# Chapter 10. 데이터 집계와 그룹 연산

- pandas는 데이터 집합을 자연스럽게 나누고 요약할 수 있는 groupby라는 유연한 방법을 제공함.
- 파이썬과 pandas의 강력한 표현력을 잘 이용하면 아주 복잡한 그룹 연산도 pandas 객체나 NumPy 배열을 받는 함수의 조합으로 해결할 수 있음.
- 해당 Chapter에서는 다음과 같은 내용을 배우게 된다.
    - 하나 이상의 키(함수, 배열, DataFrame의 컬럼 이름)를 이용해서 pandas 객체를 여러 조각으로 나누는 방법
    - 합계, 평균, 표준편차, 사용자 정의 함수 같은 그룹 요약 통계를 계산하는 방법
    - 정규화, 선형회귀, 등급 또는 부분집합 선택 같은 집단 내 변형이나 다른 조작을 적용하는 방법
    - 피벗테이블과 교차일람표를 구하는 방법
    - 변위치 분석과 다른 통게 집단 분석을 수행하는 방법

## 10.1 GroupBy 메카닉

- 그룹 연산은 `분리-적용-결합`으로 이해하면 편하다.
- 각 그룹의 색인은 다음과 같이 다양한 형태가 될 수 있으며, 모두 같은 타입일 필요도 없다.
    - 그룹을 묶을 축과 동일한 길이의 리스트나 배열
    - DataFrame의 컬럼 이름을 지칭하는 값
    - 그룹으로 묶을 값과 그룹 이름에 대응하는 사전이나 Series 객체
    - 축 색인 혹은 색인 내의 개별 이름에 대해 실행되는 함수
- 위 목록에서 마지막 세 방법은 객체를 나눌 때 사용할 배열을 생성하기 위한 방법임을 기억하자.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'key1' : ['a','a','b','b','a'],
                   'key2' : ['one','two','one','two','one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-1.056219,-1.352093
1,a,two,0.918420,0.607255
2,b,one,-0.339307,1.133441
3,b,two,-0.357767,-0.492343
4,a,one,-0.674555,-0.934975


> 이 데이터를 key1으로 묶고 각 그룹에서 data1의 평균을 구해보자.   
> 여러가지 방법이 있지만 그 중 하나는 data1에 대해 groupby 메서드를 호출하고 key1 컬럼을 넘기는 것이다.

In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
grouped.mean()

key1
a   -0.270785
b   -0.348537
Name: data1, dtype: float64

> 만약 여러 개의 배열을 리스트로 넘겼다면 조금 다른 결과를 얻을 수 있다.

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.865387
      two     0.918420
b     one    -0.339307
      two    -0.357767
Name: data1, dtype: float64

In [6]:
means.unstack()

key2,one,two
key1,,
a,-0.865387,0.918420
b,-0.339307,-0.357767


> 길이만 같다면 그룹의 색인은 어떤 배열이라도 상관없다.

In [7]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states, years]).mean()

California  2005    0.918420
            2006   -0.339307
Ohio        2005   -0.706993
            2006   -0.674555
Name: data1, dtype: float64

> 한 그룹으로 묶을 정보는 주로 같은 DataFrame 안에서 찾게 되는데, 이 경우 컬럼 이름(문자열, 숫자 혹은 다른 파이썬 객체)을 넘겨서 그룹의 색인으로 사용할 수 있음.  

In [8]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.270785,-0.559938
b,-0.348537,0.320549


> 위 결과를 보면 key2 컬럼이 결과에서 빠져 있는 것을 확인할 수 있다. 그 이유는 df['key2']는 숫자 데이터가 아니기 때문임.  
> 이런 컬럼은 성가신 컬럼 (nuisance column)이라고 부르며 결과에서 제외시킴.

In [9]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one  -0.865387 -1.143534
     two   0.918420  0.607255
b    one  -0.339307  1.133441
     two  -0.357767 -0.492343

> groupby를 쓰는 목적과 별개로, 일반적으로 유용한 GroupBy 메서드는 `그룹의 크기`를 담고 있는 Series를 반환하는 Size 메서드다.

In [10]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 10.1.1 그룹 간 순회하기

> GroupBy 객체는 이터레이션을 지원하는데, 그룹 이름과 그에 따른 데이터 묶음을 튜플로 반환함.

In [11]:
for name, group in df.groupby('key1') : 
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -1.056219 -1.352093
1    a  two  0.918420  0.607255
4    a  one -0.674555 -0.934975
b
  key1 key2     data1     data2
2    b  one -0.339307  1.133441
3    b  two -0.357767 -0.492343


> 색인이 여럿 존재하는 경우 튜플의 첫 번째 원소가 색인값이 됨.

In [12]:
for (k1,k2), group in df.groupby(['key1','key2']) :
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -1.056219 -1.352093
4    a  one -0.674555 -0.934975
('a', 'two')
  key1 key2    data1     data2
1    a  two  0.91842  0.607255
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.339307  1.133441
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.357767 -0.492343


> 한 줄이면 그룹별 데이터를 사전형으로 쉽게 바꿔서 유용하게 사용할 수 있음.

In [13]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.339307,1.133441
3,b,two,-0.357767,-0.492343


> groupby 메서드는 기본적으로 axis = 0 에 대해 그룹을 만드는데, 다른 축으로 그룹을 만드는 것도 가능.  
> 예를 들어 예제로 살펴본 df의 컬럼을 dtype에 따라 그룹으로 묶을 수도 있음.

In [15]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [18]:
grouped = df.groupby(df.dtypes, axis = 1)

for dtype, group in grouped :
    print(dtype)
    print(group)

float64
      data1     data2
0 -1.056219 -1.352093
1  0.918420  0.607255
2 -0.339307  1.133441
3 -0.357767 -0.492343
4 -0.674555 -0.934975
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 10.1.2 컬럼이나 컬럼의 일부만 선택하기

> DataFrame에서 만든 GroupBy 객체를 컬럼 이름이나 컬럼 이름이 담긴 배열로 색인하면 수집을 위해 해당 컬럼을 선택하게 됨.  
> 상단에 위치한 코드는 아래 코드의 신택틱 슈거로 두개 다 같은 값을 반환함.

In [27]:
df.groupby('key1')['data1'].mean()

key1
a   -0.270785
b   -0.348537
Name: data1, dtype: float64

In [28]:
df['data1'].groupby(df['key1']).mean()

key1
a   -0.270785
b   -0.348537
Name: data1, dtype: float64

> 대용량 데이터를 다룰 경우 소수의 컬럼만 집계하고 싶고 결과를 DataFrame으로 받고 싶다면 아래와 같이 작성한다.

In [29]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one  -1.143534
     two   0.607255
b    one   1.133441
     two  -0.492343

> 색인으로 얻은 객체는 groupby 메서드에 `리스트`나 `배열`을 넘겼을 경우 `DataFrameGroupBy` 객체가 됨.  
> 색인으로 얻은 객체 groupby 메서드에 단일 값으로 `하나의 컬럼 이름`만 넘겼을 경우 `SeriesGroupBy` 객체가 됨.

In [32]:
s_grouped = df.groupby(['key1','key2'])['data2']   # 하나의 컬럼 이름만 넘기는 경우 Series 객체가 됨
s_grouped             

In [34]:
s_grouped.mean()

key1  key2
a     one    -1.143534
      two     0.607255
b     one     1.133441
      two    -0.492343
Name: data2, dtype: float64

In [36]:
d_grouped = df.groupby(['key1','key2'])[['data2']]  # 리스트를 넘기는 경우 DataFrame 객체가 됨
d_grouped

In [37]:
d_grouped.mean()

data2
key1 key2          
a    one  -1.143534
     two   0.607255
b    one   1.133441
     two  -0.492343

### 10.1.3 사전과 Series에서 그룹핑하기

> 그룹 정보는 배열이 아닌 형태로 존재하기도 함.

In [45]:
people = pd.DataFrame(np.random.randn(5,5),
                      columns = list('abcde'),
                      index = ['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people.iloc[2:3, [1,2]] = np.nan  # nan 값을 추가

people

,a,b,c,d,e
Joe,-0.667757,-0.161550,-2.624866,0.794498,-0.331984
Steve,0.262412,0.694519,0.712741,0.142722,0.116113
Wes,0.616616,NaN,NaN,-1.180413,0.220806
Jim,-0.277209,-0.772266,-1.094085,-0.766731,-1.728090
Travis,0.346855,-0.511015,0.883641,1.244979,-0.686450


> 이제 각 컬럼을 나타낼 그룹 목록이 있고, 그룹별로 컬럼의 값을 모두 더한다고 해보자.

In [47]:
mapping = {'a' : 'red', 
           'b' : 'red', 
           'c' : 'blue', 
           'd' : 'blue', 
           'e' : 'red',
           'f' : 'orange'}

> 이 사전에서 groupby 메서드로 넘길 배열을 뽑아낼 수 있지만 그냥 이 사전을 groupby 메서드로 넘기자.  
> (사용하지 않는 그룹 키도 문제없다는 것을 보이기 위해 'f'도 포함.)

In [48]:
by_column = people.groupby(mapping, axis = 1)
by_column.sum()

,blue,red
Joe,-1.830368,-1.161291
Steve,0.855463,1.073044
Wes,-1.180413,0.837423
Jim,-1.860816,-2.777565
Travis,2.128621,-0.850610


> Series에 대해서도 같은 기능을 수행할 수 잇는데, 고정된 크기의 맵이라고 보면 됨.

In [49]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [50]:
people.groupby(map_series, axis = 1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 10.1.4 함수로 그룹핑하기

> 파이썬 함수를 사용하는 것은 사전이나 Series를 사용해서 그룹을 매핑하는 것보다 좀 더 일반적인 방법임.  
> 그룹 색인으로 넘긴 함수는 색인값 하나마다 한 번씩 호출되며, 반환값은 그 그룹의 이름으로 사용됨.  
> 예제 people에서는 DataFrame의 색인값을 사람의 이름으로 사용했는데, 만약 이름의 길이별로 그룹을 묶고 싶다면 이름의 길이가 담긴 배열을 만들어 넘기는 대신 len 함수를 넘기면 됨.

In [51]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.328349,-0.933816,-3.718951,-1.152646,-1.839268
5,0.262412,0.694519,0.712741,0.142722,0.116113
6,0.346855,-0.511015,0.883641,1.244979,-0.686450


> 내부적으로는 모두 배열로 변환되므로 함수를 배열, 사전 또는 Series와 섞어 쓰더라도 전혀 문제가 되지 않음.  

In [52]:
key_list = ['one','one','one','two','two']

people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.667757 -0.161550 -2.624866 -1.180413 -0.331984
  two -0.277209 -0.772266 -1.094085 -0.766731 -1.728090
5 one  0.262412  0.694519  0.712741  0.142722  0.116113
6 two  0.346855 -0.511015  0.883641  1.244979 -0.686450

### 10.1.5 색인 단계로 그룹핑하기

> 계층적으로 색인된 데이터는 축 색인 단계 중 하나를 사용해서 편리하게 집계할 수 있는 기능을 제공함.

In [61]:
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],
                                     [1,3,5,1,3]],
                                     names = ['cty','tenor'])

hier_df = pd.DataFrame(np.random.randn(4,5),
                       columns = columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.050830  0.766557 -1.093277  0.975433 -0.754714
1     -0.818050  0.206955 -0.578485 -0.263617 -0.190442
2     -0.017335 -1.186706  2.260613  0.371841 -0.861071
3      1.814582  0.481880 -0.066682 -1.743004 -1.063897

> 이 기능을 사용하려면 level 예약어를 사용해서 레벨 번호나 이름을 넘기면 됨.

In [63]:
hier_df.groupby(level = 'cty', axis = 1).mean()

cty,JP,US
0,0.110360,-0.091963
1,-0.227029,-0.396526
2,-0.244615,0.352190
3,-1.403450,0.743260
